In [35]:
import redis
import torndb
import simplejson as json
import random


def date_handler(obj):
    if hasattr(obj, 'isoformat'):
        return obj.isoformat()
    else:
        raise TypeError

# pool = redis.ConnectionPool(host='192.168.55.121', port=6379, password='foobar', decode_responses=True)
pool = redis.ConnectionPool(host='123.206.254.42', port=6379, password='#q0AakIO8u*MIG*e', decode_responses=True)


db = torndb.Connection("123.206.254.42:3306", "jinshenxiangmu",
                       user="tong", password="DeepDarkM")
# torndb.pytorndb.py
r = redis.Redis(connection_pool=pool)




In [3]:
# 设置项目详情
sql = 'SELECT userID, project.projectID,projectSerial,projectName,projectBudgetSum,projectStartTime,projectEndTime,projectStatus,projectComment,projectDelete,budgetID,username,departmentName,projectDepartmentID  FROM project left JOIN department ON projectDepartmentID=departmentID left JOIN `user` ON projectChargerID=userID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Project:ID:"+str(i.get("projectID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

Project:ID:1{"userID": 2, "projectID": 1, "projectSerial": "\u9879\u76ee\u7f16\u53f71", "projectName": "\u9879\u76ee\u540d\u79f01", "projectBudgetSum": 10000, "projectStartTime": "2018-07-18", "projectEndTime": "2018-08-18", "projectStatus": 2, "projectComment": "\u5907\u6ce823333333", "projectDelete": 0, "budgetID": null, "username": "tong", "departmentName": "\u90e8\u95e81", "projectDepartmentID": "1"}
Project:ID:2{"userID": 2, "projectID": 2, "projectSerial": "\u9879\u76ee\u7f16\u53f72", "projectName": "\u9879\u76ee\u540d\u79f02", "projectBudgetSum": 10000, "projectStartTime": "2018-07-17", "projectEndTime": "2018-08-17", "projectStatus": 2, "projectComment": "\u5907\u6ce8222222222222", "projectDelete": 0, "budgetID": 1, "username": "tong", "departmentName": "\u90e8\u95e81", "projectDepartmentID": "1"}
Project:ID:3{"userID": 2, "projectID": 3, "projectSerial": "\u9879\u76ee\u7f16\u53f73", "projectName": "\u9879\u76ee\u540d\u79f03", "projectBudgetSum": 10000, "projectStartTime": "201

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [4]:
# 设置项目列表
# r.zadd("Project:All:Default",)
sql = 'SELECT project.projectID from project'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = str(i.get("projectID"))
    key = "Project:All:Default"
#     print (key+value)
    r.zadd(key, i.get("projectID"), value)
pipe.execute()

[]

In [5]:
# 测试获取项目列表
prolist = r.zrange("Project:All:Default", 1, 2)
result=[]
keys=[]
for i in prolist:
    key = "Project:ID:"+i
    result.append(json.loads(r.get(key)))
print(result)

[{'userID': 2, 'projectID': 2, 'projectSerial': '项目编号2', 'projectName': '项目名称2', 'projectBudgetSum': 10000, 'projectStartTime': '2018-07-17', 'projectEndTime': '2018-08-17', 'projectStatus': 2, 'projectComment': '备注222222222222', 'projectDelete': 0, 'budgetID': 1, 'username': 'tong', 'departmentName': '部门1', 'projectDepartmentID': '1'}, {'userID': 2, 'projectID': 3, 'projectSerial': '项目编号3', 'projectName': '项目名称3', 'projectBudgetSum': 10000, 'projectStartTime': '2018-07-16', 'projectEndTime': '2018-08-16', 'projectStatus': 1, 'projectComment': '备注1', 'projectDelete': 0, 'budgetID': 2, 'username': 'tong', 'departmentName': '部门1', 'projectDepartmentID': '1'}]


In [6]:
r.get("Project:ID:1")

'{"userID": 2, "projectID": 1, "projectSerial": "\\u9879\\u76ee\\u7f16\\u53f71", "projectName": "\\u9879\\u76ee\\u540d\\u79f01", "projectBudgetSum": 10000, "projectStartTime": "2018-07-18", "projectEndTime": "2018-08-18", "projectStatus": 2, "projectComment": "\\u5907\\u6ce823333333", "projectDelete": 0, "budgetID": null, "username": "tong", "departmentName": "\\u90e8\\u95e81", "projectDepartmentID": "1"}'

In [7]:
# 所有用户列表
sql = 'SELECT userID from user'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = str(i.get("userID"))
    key = "User:All:Default"
#     print (key+value)
    r.zadd(key,value, value)
pipe.execute()

[]

In [8]:
# 用户的项目列表
userList = r.zrange("User:All:Default", 0 ,-1)
sql = 'SELECT project.projectSerial,project.projectID from project where projectChargerID = %s'
pipe = r.pipeline(transaction=True)
for j in userList:
    result = db.query(sql, j)
    for i in result:
        value = str(i.get("projectSerial"))
        key = "ProjectSerial:User:"+j
        pipe.zadd(key, value,i.get("projectID") )
pipe.execute()

[0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1]

In [9]:
# 用户的项目下的合同列表
userList = r.zrange("User:All:Default", 0 ,-1)
sql = 'SELECT contractSerial,contractID FROM contract WHERE projectID= %s'
pipe = r.pipeline(transaction=True)
for j in userList:
    for i in r.zrange("Project:User:"+j,0,-1):
        for k in db.query(sql,i):
            value = str(k.get("contractSerial"))
            key1 = "ContractSerial:User:"+j+":Project:"+i
            key2 = "ContracSerialt:User:"+j
            pipe.zadd(key1, value, k.get("contractID"), )
            pipe.zadd(key2, value, k.get("contractID"))


pipe.execute()
        


[1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [10]:
# 设置合同详情
sql = 'SELECT contractID,contractSerial,contractName,contractPartner,contractSignedTime,contractMoney,contractComment,projectSerial,contractDelete,contractOwner,contractContent FROM contract INNER JOIN project ON project.projectID = contract.projectID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Contract:ID:"+str(i.get("contractID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

Contract:ID:235{"contractID": 235, "contractSerial": "\u5408\u540c\u7f16\u53f710", "contractName": "\u5408\u540c\u540d\u79f010", "contractPartner": "\u7532\u3001\u4e59\u3001\u4e19\u3001\u4e01", "contractSignedTime": "2018-08-01", "contractMoney": 10000, "contractComment": "\u5907\u6ce810", "projectSerial": "\u9879\u76ee\u7f16\u53f71", "contractDelete": 0, "contractOwner": null, "contractContent": "xxx10"}
Contract:ID:236{"contractID": 236, "contractSerial": "\u5408\u540c\u7f16\u53f71", "contractName": "\u5408\u540c\u540d\u79f01", "contractPartner": "\u7532\u4e59\u4e19", "contractSignedTime": "2018-07-17", "contractMoney": 200, "contractComment": "\u5907\u6ce81", "projectSerial": "\u9879\u76ee\u7f16\u53f71", "contractDelete": 0, "contractOwner": null, "contractContent": "xxx1"}
Contract:ID:237{"contractID": 237, "contractSerial": "\u5408\u540c\u7f16\u53f73", "contractName": "\u5408\u540c\u540d\u79f03", "contractPartner": "\u7532\u4e59\u4e19", "contractSignedTime": "2018-07-15", "contrac

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [11]:
# 设置文档详情
sql = 'SELECT documentID, documentSerial, documentName, documentUploadTime, documentOwner , documentComment, documentUrl, documentType, documentDelete, projectSerial , contractSerial FROM document INNER JOIN project ON document.`projectID` = project.`projectID` INNER JOIN contract ON document.contractID = contract.contractID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Document:ID:"+str(i.get("documentID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

Document:ID:2{"documentID": 2, "documentSerial": "2", "documentName": "\u6587\u68632", "documentUploadTime": "2018-07-16", "documentOwner": "qiang", "documentComment": "xx", "documentUrl": "", "documentType": 0, "documentDelete": 0, "projectSerial": "\u9879\u76ee\u7f16\u53f72", "contractSerial": "\u5408\u540c\u7f16\u53f73"}
Document:ID:3{"documentID": 3, "documentSerial": "3", "documentName": "\u6587\u68633", "documentUploadTime": "2018-07-17", "documentOwner": "wan", "documentComment": "xx", "documentUrl": "", "documentType": 0, "documentDelete": 0, "projectSerial": "\u9879\u76ee\u7f16\u53f73", "contractSerial": "\u5408\u540c\u7f16\u53f75"}
Document:ID:4{"documentID": 4, "documentSerial": "4", "documentName": "\u6587\u68634", "documentUploadTime": "2018-07-15", "documentOwner": "peng", "documentComment": "xx", "documentUrl": "", "documentType": 0, "documentDelete": 0, "projectSerial": "\u9879\u76ee\u7f16\u53f74", "contractSerial": "\u5408\u540c\u7f16\u53f76"}


[True, True, True]

In [12]:
# 设置文档详情
sql = 'SELECT documentID, documentSerial, documentName, documentUploadTime, documentOwner , documentComment, documentUrl, documentType, documentDelete, projectSerial , contractSerial FROM document INNER JOIN project ON document.`projectID` = project.`projectID` INNER JOIN contract ON document.contractID = contract.contractID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Document:ID:"+str(i.get("documentID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

Document:ID:2{"documentID": 2, "documentSerial": "2", "documentName": "\u6587\u68632", "documentUploadTime": "2018-07-16", "documentOwner": "qiang", "documentComment": "xx", "documentUrl": "", "documentType": 0, "documentDelete": 0, "projectSerial": "\u9879\u76ee\u7f16\u53f72", "contractSerial": "\u5408\u540c\u7f16\u53f73"}
Document:ID:3{"documentID": 3, "documentSerial": "3", "documentName": "\u6587\u68633", "documentUploadTime": "2018-07-17", "documentOwner": "wan", "documentComment": "xx", "documentUrl": "", "documentType": 0, "documentDelete": 0, "projectSerial": "\u9879\u76ee\u7f16\u53f73", "contractSerial": "\u5408\u540c\u7f16\u53f75"}
Document:ID:4{"documentID": 4, "documentSerial": "4", "documentName": "\u6587\u68634", "documentUploadTime": "2018-07-15", "documentOwner": "peng", "documentComment": "xx", "documentUrl": "", "documentType": 0, "documentDelete": 0, "projectSerial": "\u9879\u76ee\u7f16\u53f74", "contractSerial": "\u5408\u540c\u7f16\u53f76"}


[True, True, True]

In [13]:
# 设置资产详情
sql = 'SELECT assetID, assetSerial, assetName, assetSpec, assetModel , assetAmount, assetMoney, assetDate, assetOwner, assetComment , assetStatus, assetDelete, projectSerial, assetDepID FROM asset INNER JOIN project ON asset.projectID = project.projectID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Asset:ID:"+str(i.get("assetID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

Asset:ID:1{"assetID": 1, "assetSerial": "zichanbiaohao1", "assetName": "zichanname1", "assetSpec": "guige1", "assetModel": "xinghao1", "assetAmount": 1000, "assetMoney": 100000, "assetDate": "2018-07-24", "assetOwner": "tong", "assetComment": "beizhu1", "assetStatus": 0, "assetDelete": 0, "projectSerial": "\u9879\u76ee\u7f16\u53f72", "assetDepID": "1"}
Asset:ID:2{"assetID": 2, "assetSerial": "zichanbiaohao2", "assetName": "zichanname2", "assetSpec": "guige2", "assetModel": "xinghao2", "assetAmount": 200, "assetMoney": 20000, "assetDate": "2018-07-24", "assetOwner": "qiang", "assetComment": "beizhu2", "assetStatus": 0, "assetDelete": 0, "projectSerial": "\u9879\u76ee\u7f16\u53f72", "assetDepID": "2"}
Asset:ID:3{"assetID": 3, "assetSerial": "zichanbiaohao3", "assetName": "zichanname3", "assetSpec": "guige3", "assetModel": "xinghao3", "assetAmount": 300, "assetMoney": 30000, "assetDate": "2018-07-23", "assetOwner": "wan", "assetComment": "beizhu3", "assetStatus": 0, "assetDelete": 0, "pro

[True, True, True, True, True, True, True]

In [14]:
# 设置财务详情
sql = 'SELECT financeID,financeSerials,financeDate,financeName,financeMoney,financeStatus,financeDelete,borrowName,lendName,buyOrgFormInfo,buyMethodInfo,buyTypeInfo,payMethodInfo,projectSerial,financeBorrowerID,financeLenderID,financeTypeID,financeBuyOrgID,financeBuyMethodID,finance.payMethodID,contractSerial FROM finance INNER JOIN ( SELECT accountantID,accountantName AS borrowName FROM accountant) borrow ON financeBorrowerID=borrow.accountantID INNER JOIN ( SELECT accountantID,accountantName AS lendName FROM accountant) lend ON financeLenderID=lend.accountantID INNER JOIN buyorgform ON financeBuyOrgID=buyOrgFormID INNER JOIN buymethod ON financeBuyMethodID=buyMethodID INNER JOIN buytype ON financeTypeID=buyTypeID INNER JOIN paymethod ON finance.payMethodID=paymethod.`payMethodID` INNER JOIN project ON finance.`projectID`=project.`projectID` INNER JOIN contract ON finance.`contractID`=contract.`contractID`'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Finance:ID:"+str(i.get("financeID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

Finance:ID:4{"financeID": 4, "financeSerials": "4", "financeDate": "2018-07-13", "financeName": "fiance4", "financeMoney": 40000, "financeStatus": 0, "financeDelete": 0, "borrowName": "\u8d22\u52a13", "lendName": "\u8d22\u52a14", "buyOrgFormInfo": "buyorgform4", "buyMethodInfo": "buymethod1", "buyTypeInfo": "buytype4", "payMethodInfo": "info4", "projectSerial": "\u9879\u76ee\u7f16\u53f71", "financeBorrowerID": "3", "financeLenderID": "4", "financeTypeID": "4", "financeBuyOrgID": "4", "financeBuyMethodID": "1", "payMethodID": "4", "contractSerial": "\u5408\u540c\u7f16\u53f79"}
Finance:ID:2{"financeID": 2, "financeSerials": "2", "financeDate": "2018-07-11", "financeName": "fiance2", "financeMoney": 20000, "financeStatus": 0, "financeDelete": 0, "borrowName": "\u8d22\u52a18", "lendName": "\u8d22\u52a14", "buyOrgFormInfo": "buyorgform2", "buyMethodInfo": "buymethod3", "buyTypeInfo": "buytype1", "payMethodInfo": "info4", "projectSerial": "\u9879\u76ee\u7f16\u53f71", "financeBorrowerID": "8"

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [15]:
# 设置预算详情
sql = 'SELECT budgetdetail.budgetID,budgetSerial,project.projectBudgetSum,projectContractsSum,projectContractsPayed,projectContractsNotPayed,projectBudgetLeft,project.projectID,projectSerial,projectName,projectStartTime FROM budgetdetail INNER JOIN project ON budgetdetail.projectID=project.projectID'
result = db.query(sql)
pipe = r.pipeline(transaction=True)
for i in result:
    value = json.dumps(i, default=date_handler)
    key = "Budget:ID:"+str(i.get("budgetID"))
    print (key+value)
    pipe.set(key, value)
pipe.execute()

Budget:ID:1{"budgetID": 1, "budgetSerial": "\u9884\u7b97\u7f16\u53f71", "projectBudgetSum": 10000, "projectContractsSum": 0, "projectContractsPayed": 111111, "projectContractsNotPayed": 4400, "projectBudgetLeft": 0, "projectID": 3, "projectSerial": "\u9879\u76ee\u7f16\u53f73", "projectName": "\u9879\u76ee\u540d\u79f03", "projectStartTime": "2018-07-16"}
Budget:ID:2{"budgetID": 2, "budgetSerial": "\u9884\u7b97\u7f16\u53f72", "projectBudgetSum": 10000, "projectContractsSum": 80, "projectContractsPayed": 22302, "projectContractsNotPayed": 4400, "projectBudgetLeft": -80, "projectID": 2, "projectSerial": "\u9879\u76ee\u7f16\u53f72", "projectName": "\u9879\u76ee\u540d\u79f02", "projectStartTime": "2018-07-17"}
Budget:ID:3{"budgetID": 3, "budgetSerial": "3", "projectBudgetSum": 10000, "projectContractsSum": 0, "projectContractsPayed": 0, "projectContractsNotPayed": 0, "projectBudgetLeft": 0, "projectID": 1, "projectSerial": "\u9879\u76ee\u7f16\u53f71", "projectName": "\u9879\u76ee\u540d\u79f0

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [16]:
# 查 选择借方信息 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  借方下拉框
sql2= "SELECT accountantID,accountantName,accountantType,versionID FROM accountant inner join version on accountant.versionID = version.verisionID WHERE accountantType = 0 and versionBuildNum = %s"
resultHtml = '<option value="">请选择贷方信息</option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'


    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["accountantID"] +'">' + j["accountantName"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "BorrowType:list"
   
r.set(key, resultHtml)
    
   

<option value="">请选择贷方信息</option><optgroup label="v4.0"><option value="8">财务8</option></optgroup><optgroup label="v3.0（已失效）"><option value="3">财务3</option></optgroup><optgroup label="v2.0（已失效）"><option value="2">财务2</option></optgroup><optgroup label="v1.0（已失效）"><option value="1">财务1</option><option value="7">财务7</option></optgroup>


True

In [17]:
# 查 贷方下拉框 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  贷方下拉框
sql2= " SELECT accountantID,accountantName,accountantType,versionID FROM accountant inner join version on accountant.versionID = version.verisionID WHERE accountantType = 1 and versionBuildNum = %s"
resultHtml = '<option value="">请选择贷方信息</option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
        
        
 
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["accountantID"] +'">' + j["accountantName"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "LendType:list"
   
r.set(key, resultHtml)

<option value="">请选择贷方信息</option><optgroup label="v4.0"><option value="4">财务4</option></optgroup><optgroup label="v3.0（已失效）"><option value="6">财务6</option></optgroup><optgroup label="v2.0（已失效）"><option value="5">财务5</option></optgroup><optgroup label="v1.0（已失效）"></optgroup>


True

In [18]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  预算批复下拉框
sql2= " SELECT budgetReplyID,budgetReplyInfo,versionID FROM budgetreply inner join version on budgetreply.versionID = version.verisionID WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["budgetReplyID"] +'">' + j["budgetReplyInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "BudgetReply:list"
   
r.set(key, resultHtml)

<option value=""></option><optgroup label="v4.0"></optgroup><optgroup label="v3.0（已失效）"><option value="2">批复2</option></optgroup><optgroup label="v2.0（已失效）"><option value="">pifu2</option><option value="4">批复4</option></optgroup><optgroup label="v1.0（已失效）"><option value="1">批复1</option><option value="3">批复3</option></optgroup>


True

In [19]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  建设内容下拉框
sql2= " SELECT buildContentID,buildInfo,versionID FROM buildcontent inner join version on buildcontent.versionID = version.verisionID WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["buildContentID"] +'">' + j["buildInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "BuildContent:list"
   
r.set(key, resultHtml)

<option value=""></option><optgroup label="v4.0"><option value="4">建设内容4</option></optgroup><optgroup label="v3.0（已失效）"><option value="3">建设内容3</option></optgroup><optgroup label="v2.0（已失效）"><option value="2">建设内容2</option></optgroup><optgroup label="v1.0（已失效）"><option value="1">建设内容1</option></optgroup>


True

In [20]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  采购方式下拉框
sql2= " SELECT buyMethodID,buyMethodInfo,versionID FROM buymethod inner join version on buymethod.versionID = version.verisionID WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["buyMethodID"] +'">' + j["buyMethodInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "PurchaseWay:list"
   
r.set(key, resultHtml)

<option value=""></option><optgroup label="v4.0"><option value="4">buymethod4</option></optgroup><optgroup label="v3.0（已失效）"><option value="3">buymethod3</option></optgroup><optgroup label="v2.0（已失效）"><option value="2">buymethod2</option></optgroup><optgroup label="v1.0（已失效）"><option value="1">buymethod1</option></optgroup>


True

In [21]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  采购组织方式下拉框
sql2= " SELECT buyOrgFormID,buyOrgFormInfo,versionID FROM buyorgform inner join version on buyorgform.versionID = version.verisionID  WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2,versionBuildNum):
        resultHtml += '<option value="' + j["buyOrgFormID"] +'">' + j["buyOrgFormInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "BuyOrgMethod:list"
   
r.set(key, resultHtml)

<option value=""></option><optgroup label="v4.0"><option value="4">buyorgform4</option></optgroup><optgroup label="v3.0（已失效）"><option value="3">buyorgform3</option></optgroup><optgroup label="v2.0（已失效）"><option value="2">buyorgform2</option></optgroup><optgroup label="v1.0（已失效）"><option value="1">buyorgform1</option></optgroup>


True

In [22]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  采购类型下拉框
sql2= " SELECT buyTypeID,buyTypeInfo,versionID FROM buytype inner join version on buytype.versionID = version.verisionID  WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["buyTypeID"] +'">' + j["buyTypeInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "PurchaseType:list"
   
r.set(key, resultHtml)

<option value=""></option><optgroup label="v4.0"><option value="4">buytype4</option></optgroup><optgroup label="v3.0（已失效）"><option value="3">buytype3</option></optgroup><optgroup label="v2.0（已失效）"><option value="2">buytype2</option></optgroup><optgroup label="v1.0（已失效）"><option value="1">buytype1</option></optgroup>


True

In [23]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  部门下拉框
sql2= " SELECT departmentID,departmentName,versionID FROM department inner join version on department.versionID = version.verisionID  WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["departmentID"] +'">' + j["departmentName"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "Department:list"
   
r.set(key, resultHtml)

<option value=""></option><optgroup label="v4.0"></optgroup><optgroup label="v3.0（已失效）"><option value="4">部门4</option></optgroup><optgroup label="v2.0（已失效）"><option value="3">部门3</option></optgroup><optgroup label="v1.0（已失效）"><option value="1">部门1</option><option value="2">部门2</option></optgroup>


True

In [24]:
# 查 version  
sql1 = "select versionBuildNum from version order by verisionID desc "
versionList = []
for i in db.query(sql1):
    versionList.append(i["versionBuildNum"])
    
# 查具体的  付款方式下拉框
sql2= " SELECT payMethodID,payMethodInfo,versionID FROM paymethod inner join version on paymethod.versionID = version.verisionID  WHERE versionBuildNum = %s"
resultHtml = '<option value=""></option>'
i=0
for versionBuildNum in versionList:
    if i==0:
        resultHtml += '<optgroup label="'+ versionBuildNum +'">'
    else:
        resultHtml += '<optgroup label="'+ versionBuildNum +'（已失效）">'
    for j in  db.query(sql2, versionBuildNum):
        resultHtml += '<option value="' + j["payMethodID"] +'">' + j["payMethodInfo"] + '</option>'
    resultHtml += '</optgroup>'
    i+=1

# resultHtml += '</select>'

print(resultHtml)


# value = json.dumps(resultList)
key = "PayMethod:list"
   
r.set(key, resultHtml)

<option value=""></option><optgroup label="v4.0"><option value="4">info4</option></optgroup><optgroup label="v3.0（已失效）"><option value="3">info3</option></optgroup><optgroup label="v2.0（已失效）"><option value="2">info2</option></optgroup><optgroup label="v1.0（已失效）"><option value="1">info1</option></optgroup>


True

In [25]:
# 权限下拉框
sql = 'SELECT permissionID,url,LEVEL,permissionComment FROM permission'
result = db.query(sql)
value = json.dumps(i, default=date_handler)
key = "Level:list"
r.set(key, value)

True

In [26]:
# 项目钱数相关缓存

sql ='select projectID from project'
projectList = []
for i in db.query(sql):
    projectList.append(i["projectID"])
sql2 = 'select contractMoney from contract where projectID = %s'

pipe = r.pipeline(transaction=True)
for projectID in projectList:
    contractSum = 0
    for contract in db.query(sql2, projectID):
        contractSum += contract["contractMoney"]
#     print(contractSum, projectID)
    key = 'Project:'+str(projectID)+':Contract:Sum'
    
    pipe.set(key, contractSum)
pipe.execute()

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [27]:
r.get("Project:3:Contract:Sum")

'0'

In [28]:
# 项目钱数相关缓存

sql ='select contractID, contractMoney from contract'
contractList = []
pipe = r.pipeline(transaction=True)
for i in db.query(sql):
    contractList.append(i["contractID"])
    pipe.set( 'Contract:ID:'+str(i["contractID"])+":AllMoney", i["contractMoney"])
#     print(i["contractMoney"])
pipe.execute()

# contractMoneyList = []
sql = 'select sum(financeMoney) as financeMoney from finance where contractID = %s and financeStatus=1 GROUP  BY contractID'
sql2 = 'select sum(financeMoney) as financeMoney  from finance where contractID = %s and financeStatus=0 GROUP  BY contractID'


pipe = r.pipeline(transaction=True)
for i in contractList:
#     db.query(sql, i["contractID"])
    SumPay = db.query(sql, i)
    FinUnPay = db.query(sql2, i)
    
    if SumPay:
        pipe.set("Contract:ID:"+str(i)+":SumPay",  SumPay[0]['financeMoney'])
        pipe.set("Contract:ID:"+str(i)+":UnPay", int(r.get( "Contract:ID:"+str(i)+":AllMoney")) - int(SumPay[0]['financeMoney']))
    else:
        pipe.set("Contract:ID:"+str(i)+":SumPay",  0)
        pipe.set("Contract:ID:"+str(i)+":UnPay", r.get( "Contract:ID:"+str(i)+":AllMoney"))
    if FinUnPay:
        pipe.set("Contract:ID:"+str(i)+":FinUnPay", FinUnPay[0]['financeMoney'])
    else:
        pipe.set("Contract:ID:"+str(i)+":FinUnPay", 0)
    if SumPay and FinUnPay:
        pipe.set("Contract:ID:"+str(i)+":Extra", int(r.get( 'Contract:ID:'+str(i)+":AllMoney")) - int(SumPay[0]['financeMoney'] )- int(FinUnPay[0]['financeMoney']))
    else:
        pipe.set("Contract:ID:"+str(i)+":Extra", r.get( 'Contract:ID:'+str(i)+":AllMoney"))


pipe.execute()

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [29]:
print(type(input("请输入")))

KeyboardInterrupt: 

In [30]:
# 设置用户信息缓存

sql = 'select * from user '
pipe = r.pipeline(transaction=True)


for i in db.query(sql):
    key = "User:ID:"+str(i["userID"])
    value = json.dumps(i)
    pipe.set(key, value)
pipe.execute()

[True, True, True, True, True, True, True, True, True]

In [36]:
# 设置权限缓存
sql = " select * from permission"
pipe = r.pipeline(transaction=True)
for i in db.query(sql):
    pipe.sadd("Permission", i["url"]+":"+str(i["level"]))
pipe.execute()

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [32]:
import pandas as pd
from pandas import Series,DataFrame

sql = 'select username, userID from user'
userFrame = DataFrame(db.query(sql))
sql = 'select projectID, projectName as name, IFNULL(projectBudgetSum,9000)  as size, IFNULL(projectChargerID,0)  as projectChargerID, username from project inner join user on projectChargerID = userID order by projectChargerID desc'
projectFrame = DataFrame(db.query(sql))
sql ='select contractID, contractName as name, 10000 as size, projectID FROM contract order by projectID desc'
contractFrame = DataFrame(db.query(sql))
sql ='select financeID, financeName as name, financeMoney as size, contractID FROM finance order by financeID desc'
financeFrame = DataFrame(db.query(sql))
contractFrame
    
resultDict = {}
resultDict["name"] = '金审平台'
resultDict["size"] = str(18000)
resultDict["children"]=[]

for indexs in userFrame.index:
    userNode = {}
    userNode["name"] = str(userFrame.loc[indexs].values[1])
    userNode["userID"] = str(userFrame.loc[indexs].values[0])
    userNode["size"] = str(8000)
    userNode["children"] = []

    for indexs in projectFrame[projectFrame["projectChargerID"] == int(userNode["userID"])].index:
        projectNode = {}
        projectNode["name"] = str(projectFrame.loc[indexs].values[0]) 
        projectNode["size"] = str(projectFrame.loc[indexs].values[3])
        projectNode["projectID"] = str(projectFrame.loc[indexs].values[2])
        projectNode["children"] = []

        for indexs in contractFrame[contractFrame["projectID"] == int(projectNode["projectID"])].index:
            contractNode ={}
            contractNode["name"] =  str(contractFrame.loc[indexs].values[1])
            contractNode["size"] = str(contractFrame.loc[indexs].values[3])
            contractNode["contractID"] = str(contractFrame.loc[indexs].values[0])
            contractNode["children"] = []

            for indexs in financeFrame[financeFrame["contractID"] == int(contractNode["contractID"])].index:
                financeNode = {}
                financeNode["name"] =  str(financeFrame.loc[indexs].values[2])
                financeNode["size"] = str(financeFrame.loc[indexs].values[3])
    #             financeNode["children"] = []

                contractNode["children"].append(financeNode)
            projectNode["children"].append(contractNode)
        userNode["children"].append(projectNode)
    resultDict["children"].append(userNode)
json.dumps(resultDict, ensure_ascii=False)


 

'{"name": "金审平台", "size": "18000", "children": [{"name": "qiang", "userID": "1", "size": "8000", "children": [{"name": "ada", "size": "22", "projectID": "81", "children": []}, {"name": "44", "size": "122", "projectID": "71", "children": []}, {"name": "45唯一性测试", "size": "1112", "projectID": "79", "children": []}, {"name": "项目名称4", "size": "10000", "projectID": "4", "children": [{"name": "合同名称25", "size": "10000", "contractID": "278", "children": []}, {"name": "合同名称29", "size": "10000", "contractID": "282", "children": []}]}, {"name": "dada", "size": "9000", "projectID": "86", "children": []}, {"name": "45", "size": "111", "projectID": "72", "children": []}]}, {"name": "tong", "userID": "2", "size": "8000", "children": [{"name": "测试test1", "size": "400", "projectID": "87", "children": [{"name": "测试test1合同2", "size": "10000", "contractID": "299", "children": []}, {"name": "测试test1合同1", "size": "10000", "contractID": "298", "children": [{"name": "测试1合同1财务1", "size": "60"}]}]}, {"name": "项目

In [37]:
r.sismember('/getContractList:3')

TypeError: sismember() missing 1 required positional argument: 'value'

In [ ]:
|